<a href="https://colab.research.google.com/github/thwlswkdgh/GAN/blob/main/code_practice/CycleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Git clone

In [1]:
!git clone https://github.com/davidADSP/GDL_code

Cloning into 'GDL_code'...
remote: Enumerating objects: 391, done.
remote: Total 391 (delta 0), reused 0 (delta 0), pack-reused 391
Receiving objects: 100% (391/391), 22.13 MiB | 22.15 MiB/s, done.
Resolving deltas: 100% (237/237), done.


### ipynb 합치기

In [2]:
#!pip install nbmerge
##nbmerge file_1.ipynb file_2.ipynb file_3.ipynb > merged.ipynb

### Gdrive mount

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 경로 이동

In [4]:
%cd /content/drive/MyDrive/GAN/GAN2_Src

/content/drive/MyDrive/GAN/GAN2_Src


In [5]:
!pwd

/content/drive/MyDrive/GAN/GAN2_Src


In [6]:
!ls

03_01_autoencoder.ipynb      apple2orange-dataset.zip  metadata.csv
03_02_vae_digit.ipynb	     celeba-dataset	       models
03_03_vae_face.ipynb	     celeba-dataset.zip        old
05_01_cyclegan_train_.ipynb  Common.ipynb	       requirements.txt
05_01_cyclegan_train.ipynb   data		       run
apple2orange		     GDL_code		       utils


### 환경 설정(PIP install requirements)

In [7]:
!pip install -r requirements.txt

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-betkf1h7
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-betkf1h7
     |████████████████████████████████| 96 kB 3.9 MB/s 
     |████████████████████████████████| 157 kB 52.5 MB/s 
     |████████████████████████████████| 701 kB 48.0 MB/s 
     |████████████████████████████████| 10.6 MB 31.7 MB/s 
     |████████████████████████████████| 2.8 MB 43.2 MB/s 
     |████████████████████████████████| 113 kB 67.4 MB/s 
     |████████████████████████████████| 768 kB 40.0 MB/s 
     |████████████████████████████████| 111 kB 56.0 MB/s 
     |████████████████████████████████| 177 kB 59.2 MB/s 
     |████████████████████████████████| 126 kB 59.2 MB/s 
     |████████████████████████████████| 89 kB 8.4 MB/s 
     |████████████████████████████████| 126 kB 52.8 MB/s 
     |████████████████████████████████| 312 kB 52.1 MB/s 
     |████████████████████████████████| 51 kB

KAGGLE 데이터 불러오기

In [8]:
# !pip install -q kaggle
# from google.colab import files
# uploaded = files.upload()

# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 /root/.kaggle/kaggle.json
# !kaggle datasets download -d balraj98/apple2orange-dataset

In [9]:
# !unzip apple2orange-dataset.zip  -d ./apple2orange/

# CycleGAN train

In [10]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 6.9 MB/s 


In [11]:
import os
import matplotlib.pyplot as plt

from models.cycleGAN import CycleGAN
from utils.loaders import DataLoader

In [12]:

# run params
SECTION = 'paint'
RUN_ID = '0001'
DATA_NAME = 'apple2orange'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' # 'build' # 

# data

In [13]:
IMAGE_SIZE = 128

In [14]:

data_loader = DataLoader(dataset_name=DATA_NAME, img_res=(IMAGE_SIZE, IMAGE_SIZE))


# architecture

In [15]:
gan = CycleGAN(
    input_dim = (IMAGE_SIZE,IMAGE_SIZE,3)
    ,learning_rate = 0.0002
    , buffer_max_length = 50
    , lambda_validation = 1
    , lambda_reconstr = 10
    , lambda_id = 2
    , generator_type = 'unet'
    , gen_n_filters = 32
    , disc_n_filters = 32
    )

if mode == 'build':
    gan.save(RUN_FOLDER)
else:
    gan.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))
    


In [16]:
gan.g_BA.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_18 (Conv2D)             (None, 64, 64, 32)   1568        ['input_4[0][0]']                
                                                                                                  
 instance_normalization_13 (Ins  (None, 64, 64, 32)  0           ['conv2d_18[0][0]']              
 tanceNormalization)                                                                              
                                                                                            

In [17]:
gan.g_AB.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_10 (Conv2D)             (None, 64, 64, 32)   1568        ['input_3[0][0]']                
                                                                                                  
 instance_normalization_6 (Inst  (None, 64, 64, 32)  0           ['conv2d_10[0][0]']              
 anceNormalization)                                                                               
                                                                                            

In [18]:
gan.d_A.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 32)        1568      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 64, 64, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        32832     
                                                                 
 instance_normalization (Ins  (None, 32, 32, 64)       0         
 tanceNormalization)                                             
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 32, 32, 64)        0         
                                                             

In [19]:
gan.d_B.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_5 (Conv2D)           (None, 64, 64, 32)        1568      
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 64, 64, 32)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 32, 32, 64)        32832     
                                                                 
 instance_normalization_3 (I  (None, 32, 32, 64)       0         
 nstanceNormalization)                                           
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 32, 32, 64)        0         
                                                           

# train

In [20]:
BATCH_SIZE = 1
EPOCHS = 200
PRINT_EVERY_N_BATCHES = 10

TEST_A_FILE = 'n07740461_14740.jpg'
TEST_B_FILE = 'n07749192_4241.jpg'

In [ ]:
gan.train(data_loader
        , run_folder = RUN_FOLDER
        , epochs=EPOCHS
        , test_A_file = TEST_A_FILE
        , test_B_file = TEST_B_FILE
        , batch_size=BATCH_SIZE
        , sample_interval=PRINT_EVERY_N_BATCHES)
        

train!!!
[Epoch 0/200] [Batch 0/995] [D loss: 1.382527, acc:  44%] [G loss: 18.368679, adv: 2.164230, recon: 1.374835, id: 1.228050] time: 0:00:21.308040 
INFO:tensorflow:Assets written to: ram://575a5f93-19ba-4d09-ad07-b699b26d180e/assets
INFO:tensorflow:Assets written to: ram://ab6a0f33-959f-4957-9290-7c5b07a34354/assets
INFO:tensorflow:Assets written to: ram://7386aea2-4f23-47ad-b081-366f325119b0/assets
INFO:tensorflow:Assets written to: ram://109ab31a-a3a7-44e0-9a95-e82043a586b9/assets
INFO:tensorflow:Assets written to: ram://c2827579-f5a4-4435-93cf-6e110fe4fa02/assets
[Epoch 0/200] [Batch 1/995] [D loss: 1.082453, acc:  47%] [G loss: 18.180828, adv: 2.191180, recon: 1.345175, id: 1.268952] time: 0:01:00.586012 
[Epoch 0/200] [Batch 2/995] [D loss: 0.910689, acc:  47%] [G loss: 18.211939, adv: 2.230062, recon: 1.329252, id: 1.344677] time: 0:01:03.802912 
[Epoch 0/200] [Batch 3/995] [D loss: 1.281653, acc:  47%] [G loss: 14.393703, adv: 1.628410, recon: 1.023546, id: 1.264916] time

# loss

In [ ]:
fig = plt.figure(figsize=(20,10))

plt.plot([x[1] for x in gan.g_losses], color='green', linewidth=0.1) #DISCRIM LOSS
# plt.plot([x[2] for x in gan.g_losses], color='orange', linewidth=0.1)
plt.plot([x[3] for x in gan.g_losses], color='blue', linewidth=0.1) #CYCLE LOSS
# plt.plot([x[4] for x in gan.g_losses], color='orange', linewidth=0.25)
plt.plot([x[5] for x in gan.g_losses], color='red', linewidth=0.25) #ID LOSS
# plt.plot([x[6] for x in gan.g_losses], color='orange', linewidth=0.25)

plt.plot([x[0] for x in gan.g_losses], color='black', linewidth=0.25)

# plt.plot([x[0] for x in gan.d_losses], color='black', linewidth=0.25)

plt.xlabel('batch', fontsize=18)
plt.ylabel('loss', fontsize=16)

plt.ylim(0, 5)

plt.show()